In [1]:
import ibmJupyterNotebookStyles
ibmJupyterNotebookStyles.apply_ibm_styles()

## Get started with currency arbitrage optimization

In this notebook, we first define currency arbitrage and explain why the problem becomes hard and needs optimization. We later model the currency arbitrage optimization problem as a quadratic binary optimization problem and solve it using three solvers:

1. Classical solver: IBM® ILOG® CPLEX®<sup>1</sup>
2. Quantum solver: quantum approximate optimization algorithm (QAOA)<sup>2</sup>
3. Quantum solver: Gauss-Newton quantum algorithm (GNQA)<sup>3</sup>

Later, we increase the number of assets to see how the quantum solvers can solve a larger problem. 

#### Import packages

This tutorial assumes that `qiskit` and `qiskit[optimization]` packages have already been installed according to the installation instructions.

In [2]:
from qiskit import Aer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization import QuadraticProgram
from itertools import product
from operator import itemgetter
import numpy as np
from src.GNQA_StateVectorCalculation import GNQA_optimizer

### Currency arbitrage

Currency arbitrage is the process of buying and selling currencies in different markets or through different brokers in order to take advantage of price discrepancies. It involves buying a currency at a low price in one market and selling it at a higher price in another market, or buying and selling through different brokers to take advantage of differing exchange rates. For example, suppose the exchange rate for US dollars to British pounds is 1:0.8 in the US and 1:0.9 in the UK. An arbitrageur could buy dollars in the US and sell them in the UK, making a profit of 10% on the transaction. Currency arbitrage can be a risky strategy, as it requires the trader to have a deep understanding of currency markets and to be able to act quickly to take advantage of price discrepancies. It can also be difficult to find enough price discrepancies to make the strategy worthwhile.

The number of currencies involved in an arbitrage chain can be higher than two. In each financial center, the bid price is always lower than the ask price; therefore, converting money from a base currency to another and then back to base results in a loss. A longer chain of currencies, though, may result in an overall gain. An arbitrageur could potentially profit by buying dollars, converting them to pounds, and then converting the pounds to euros. However, the optimal trade would depend on the available number of currencies and the transaction costs involved between them. Finding the optimal trade, if any, becomes a hard problem by increasing the number of available currencies. 

The currency arbitrage optimization problem can be formulated as a binary optimization problem, which involves maximizing an objective function (that defines the profit from the chain of currency exchanges) subject to some constraints (having a cycle). 

Let $c_{ij}$ denote the given currency exchange rate for converting currency $i$ to currency $j$, and suppose that $c_{ij}. c_{jk}...c_{pq} c_{qi} \gt 1$ (for $q \ne p \ne ....\ne k \ne j\ne i$). Then it means that the currency conversion sequence $i \rightarrow j \rightarrow k \rightarrow ... p \rightarrow q \rightarrow i$ constitutes a currency arbitrage.<sup>4</sup>

### Data

We consider four currencies: Japanese yen (J), US dollar (U), European Union euro (E), and British pound (B). The array `c_data` has the conversion rates (convert from row to column) in that order. For example, the conversion rate from Japanese currency to US is 0.00745, while the reverse is 134.20. We will assume that these numbers include transaction costs. For the calculation of the profit, we will use the $\log_{10}$ of these conversion rates.

In [3]:
# The array cur conveys the name of the currencies
cur = ['JPY','USD','EUR','GBP']
# The following array contains conversion rate between 4 different currencies
c_data = np.array([
    [1.,        0.0074513694,   0.0073300696,   0.0061647718],
    [134.2009,  1.,             0.98370197,     0.82731795],
    [136.41706, 1.0164267,      1.,             0.84094373],
    [162.21901, 1.2086181,      1.1890916,      1.]
    ])

### Model

Our main objective is to introduce a binary programming model that has the potential to support decision-making in currency exchanges involving multiple currencies.

We build the model using `docplex`. The problem can be formulated on a directed graph $G=(V,E)$ as follows. 
Given a set $V$ of currencies that can be converted to each other with a given conversion rate (a conversion for currency $i$ to currency $j$ corresponds to an edge $(i,j) \in E$ from $i$ to $j$), we want to maximize the return of $K$ conversions (after $K$ conversions, we end with the starting currency).

- $K$ : length of the prospective closed walk

We create binary variables $x_i^k$:
$$x_i^k = \left \{ \begin{array}{ll} 1 & \text{if currency $i$ is converted in the $k$-th step} \\  
0 & \text{otherwise} 
\end{array} \right .$$
If currency $i$ is converted to currency $j$ during the $k$-th conversion, then $x_i^k x_j^{k+1} =1$. Thus, the objective function is to maximize the return, that is,<sup>4,5</sup>

\begin{equation*}
 \max \sum_{k=1}^{n} \sum_{(i,j) \in E} c_{ij} x_i^k x_j^{k+1} \tag{Objective}
\end{equation*}

and by convention we set $n+1$ conversion to be $1$ (to close the cycle). The first set of constraints needs to enforce that at each conversion step only one currency is being converted:
\begin{equation*}
\sum_{i \in V} x_i^k = 1 \quad \text{        for   }  k=1,\dots,K \tag{Constraint 1}
\end{equation*}
We also need to enforce that each currency is converted at most once:
\begin{equation*}
\sum_{k=1}^K x_i^k \le 1 \quad \text{        for   }  i \in V \tag{Constraint 2}
\end{equation*}


In [4]:
c_log_data = np.log10(c_data)
# K =< n represents the requested number of currencies in the arbitrage solution
# 
K = 3 ;
V = range(1,K+1)
    
mdl = QuadraticProgram('CAOP')

# Generating binary variables of the model
for i,k in product(cur,V):
    mdl.binary_var(name = f'x_{i}_{k}')

# Creating objective function
mdl.maximize(quadratic={(f'x_{i}_{k}',f'x_{j}_{k%K+1}'):c_log_data[cur.index(i), cur.index(j)] for i,j,k in product(cur,cur,V)})

# Creating constraints 1
for k in V:
    mdl.linear_constraint(linear={f'x_{i}_{k}':1 for i in cur}, sense='==', rhs=1)

# Creating constraints 2
for i in cur:
    mdl.linear_constraint(linear={f'x_{i}_{k}':1 for k in V}, sense='<=', rhs=1)

print(mdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CAOP

Maximize
 obj: [ - 4.255527812031 x_JPY_1*x_USD_2 + 4.255510856759 x_JPY_1*x_USD_3
      - 4.269783803309 x_JPY_1*x_EUR_2 + 4.269737371148 x_JPY_1*x_EUR_3
      - 4.420165989908 x_JPY_1*x_GBP_2 + 4.420203493269 x_JPY_1*x_GBP_3
      + 4.255510856759 x_JPY_2*x_USD_1 - 4.255527812031 x_JPY_2*x_USD_3
      + 4.269737371148 x_JPY_2*x_EUR_1 - 4.269783803309 x_JPY_2*x_EUR_3
      + 4.420203493269 x_JPY_2*x_GBP_1 - 4.420165989908 x_JPY_2*x_GBP_3
      - 4.255527812031 x_JPY_3*x_USD_1 + 4.255510856759 x_JPY_3*x_USD_2
      - 4.269783803309 x_JPY_3*x_EUR_1 + 4.269737371148 x_JPY_3*x_EUR_2
      - 4.420165989908 x_JPY_3*x_GBP_1 + 4.420203493269 x_JPY_3*x_GBP_2
      - 0.014272917750 x_USD_1*x_EUR_2 + 0.014152129581 x_USD_1*x_EUR_3
      - 0.164655105699 x_USD_1*x_GBP_2 + 0.164578187720 x_USD_1*x_GBP_3
      + 0.014152129581 x_USD_2*x_EUR_1 - 0.014272917750 x_USD_2*x_EUR_3
      + 0.164578187720 x_USD_2*x_GBP_1 -

### Result converter

Before solving the problem with various solvers, we define a function for converting the optimization result to the arbitrage result. This function takes the input of an object of the type `OptimizationResult`.<sup>6</sup>

In [5]:
# The following function is a helper function that converts the result to meaningful info and:
#              inputs:   the result, currency matrix, currency list and model
#             outputs:   the currency cycle and the final profit

def get_result_arbitrage(result, c_data, cur, model):
    raw_result = result.variables_dict
    status = model.get_feasibility_info(result.x)[0]
    cycle=None
    profit=1

    if status:
        sol = []
        for key in raw_result.keys():
            if raw_result[key] == 0: continue
            sol.append(key.split('_')[1:])
        sol.sort(key=itemgetter(1))
        n_sol = len(sol)

        cycle =''
        for idx in range(len(sol)):
            cycle += sol[idx][0]
            cycle += ' => '
        cycle += sol[0][0]

        profit = 1
        for idx in range(n_sol):
            profit *= c_data[cur.index(sol[idx][0]), cur.index(sol[(idx+1)%n_sol][0])]

    return {'cycle':cycle, 'profit':profit}

### Solve the problem with CPLEX

IBM ILOG CPLEX Optimization Studio (referred to as CPLEX) is a well-known classical optimization software package that we use as the benchmark.<sup>1</sup> We access CPLEX through Qiskit. 

In [6]:
# Calling the CplexOptimizer bundled with qiskit
cplex_result = CplexOptimizer().solve(mdl)

# Calling the helper function on results
raw_result = get_result_arbitrage(cplex_result, c_data, cur, mdl)
cycle = raw_result['cycle']
profit = raw_result['profit']

# Printing the results
print(
    '-'*80,
    '[CPLEX Result]',
    f'   optimal value: {cplex_result.fval:.5e}',
    f'   optimal solution: {cplex_result.x.astype(int)}',
    f'   status: {cplex_result.status.name}',
    f'   best arbitrage: {cycle}',
    f'   best gain: {100*(profit-1):.5f}%',
    '-'*80,
    sep='\n'
)

--------------------------------------------------------------------------------
[CPLEX Result]
   optimal value: 1.02878e-05
   optimal solution: [0 1 0 0 0 1 0 0 0 1 0 0]
   status: SUCCESS
   best arbitrage: GBP => JPY => USD => GBP
   best gain: 0.00237%
--------------------------------------------------------------------------------


### Solve the problem with quantum solvers

We first need to convert the `docplex`-based model into a quadratic unconstrained binary form using Qiskit quadratic programs<sup>6</sup> and then find the Ising-based Hamiltonian.

In [7]:
qubo = QuadraticProgramToQubo().convert(mdl)

hamiltonian_info = qubo.to_ising()

#### QAOA solver

The first quantum algorithm we use is the quadratic approximate optimization algorithm (QAOA).<sup>2</sup>

In [8]:
algorithm_globals.random_seed = 1234

# Creating the quantum instance
quantum_instance = QuantumInstance(Aer.get_backend('aer_simulator'),
                                   seed_simulator=algorithm_globals.random_seed,
                                   seed_transpiler=algorithm_globals.random_seed)

# Calling Qiskit's QAOA solver, we keep the number of repetitions to P=2
qaoa_mes = QAOA(quantum_instance=quantum_instance, reps=2)
qaoa = MinimumEigenOptimizer(qaoa_mes)  
qaoa_result = qaoa.solve(qubo)

# Calling the helper function on results
raw_result = get_result_arbitrage(qaoa_result, c_data, cur, mdl)
cycle = raw_result['cycle']
profit = raw_result['profit']

# Printing the results
print(
    '-'*80,
    '[QAOA Result]',
    f'   optimal value: {qaoa_result.fval:.5e}',
    f'   optimal solution: {qaoa_result.x.astype(int)}',
    f'   status: {qaoa_result.status.name}',
    f'   best arbitrage: {cycle}',
    f'   best gain: {100*(profit-1):.5f}%',
    '-'*80,
    sep='\n'
)

/Users/lcd/github/gbs-ibm-q/fixes/asset-lib-code-base/Algorithm Demos/Arbitrage opporutinity optimization/test-env/lib/python3.10/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lcd/github/gbs-ibm-q/fixes/asset-lib-code-base/Algorithm Demos/Arbitrage opporutinity optimization/test-env/lib/python3.10/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


--------------------------------------------------------------------------------
[QAOA Result]
   optimal value: -1.02878e-05
   optimal solution: [0 1 0 0 0 1 0 0 0 1 0 0]
   status: SUCCESS
   best arbitrage: GBP => JPY => USD => GBP
   best gain: 0.00237%
--------------------------------------------------------------------------------


#### GNQA solver

We solve the problem by using a Gauss-Newton-based quantum algorithm for combinatorial optimization (GNQA).<sup>3</sup>
For an explanation of how to execute GNQA, please refer to the Advanced Algorithm notebook for GNQA. 

Here, we use the following inverse function for the transformation of the Hamiltonian:

\begin{equation*}
    f(H) = \widehat{H}^{-p},
\end{equation*}

where $\widehat{H}$ is the normalized Hamiltonian whose spectral set is included in $\Omega = [0,1]$.

In [9]:
# definition of filter function
p = 8
filter_function = lambda x: np.power(x, -p)

# execution of GNQA
GNQA = GNQA_optimizer(qubo)
GNQA.set_filtered_hamiltonian(filter_function, Omega=[0,1], epsiron=1e-8)
GNQA_result = GNQA.solve(step_max=20)

raw_result = get_result_arbitrage(GNQA_result, c_data, cur, mdl)
cycle = raw_result['cycle']
profit = raw_result['profit']

print(
    '-'*80,
    '[GNQA Result]',
    f'   optimal value: {GNQA_result.fval:.5e}',
    f'   optimal solution: {GNQA_result.x}',
    f'   status: {GNQA_result.status.name}',
    f'   best arbitrage: {cycle}',
    f'   best gain: {100*(profit-1):.5f}%',
    '-'*80,
    sep='\n'
)

--------------------------------------------------------------------------------
[GNQA Result]
   optimal value: -1.02878e-05
   optimal solution: [0 1 0 0 0 1 0 0 0 1 0 0]
   status: SUCCESS
   best arbitrage: GBP => JPY => USD => GBP
   best gain: 0.00237%
--------------------------------------------------------------------------------


#### Interpret the solutions

As you can see, we had exactly the same results while solving the problem with CPLEX, QAOA, and GNQA. The suggested arbitrage opportunity is: GBP => JPY => USD => GBP. One can fix the starting currency (the initial investing currency) and reduce the number of variables from $nK$ to $(n-1)K$.

### Solve the problem with five assets

Now, we consider a data set with $n=5$ currencies. We want to solve the problem with a bigger size than before. 

In [10]:
# The array cur conveys the name of the currencies
cur = ['JPY','USD','CAD','CNY','EUR']
# The following array contains conversion rate between 5 different currencies
c_data = np.array([
    [1.,        0.00961,    0.01266,    0.06463,    0.00872],
    [104.05,    1.,         1.31904,    6.72585,    0.90745],
    [78.94,     0.75799,    1.,         5.10327,    0.68853],
    [15.47,     0.14864,    0.19586,    1.,         0.13488],
    [114.65,    1.10185,    1.45193,    7.41088,         1.],
    ])

We select $K=3$ to create a problem of size 15. Similarly to the previous case, the node-based formulation is:

In [11]:
c_log_data = np.log10(c_data)
K = 3
V = range(1,K+1)
    
mdl = QuadraticProgram('CAOP')

# Generating binary variables of the model
for i,k in product(cur,V):
    mdl.binary_var(name = f'x_{i}_{k}')

# Creating objective function
mdl.maximize(quadratic={(f'x_{i}_{k}',f'x_{j}_{k%K+1}'):c_log_data[cur.index(i), cur.index(j)] for i,j,k in product(cur,cur,V)})

# Creating constraints 1
for k in V:
    mdl.linear_constraint(linear={f'x_{i}_{k}':1 for i in cur}, sense='==', rhs=1)

# Creating constraints 2
for i in cur:
    mdl.linear_constraint(linear={f'x_{i}_{k}':1 for k in V}, sense='<=', rhs=1)

print(mdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CAOP

Maximize
 obj: [ - 4.034553224663 x_JPY_1*x_USD_2 + 4.034484169095 x_JPY_1*x_USD_3
      - 3.795132588637 x_JPY_1*x_CAD_2 + 3.794594244119 x_JPY_1*x_CAD_3
      - 2.379131688155 x_JPY_1*x_CNY_2 + 2.378980627399 x_JPY_1*x_CNY_3
      - 4.118967030135 x_JPY_1*x_EUR_2 + 4.118748118132 x_JPY_1*x_EUR_3
      + 4.034484169095 x_JPY_2*x_USD_1 - 4.034553224663 x_JPY_2*x_USD_3
      + 3.794594244119 x_JPY_2*x_CAD_1 - 3.795132588637 x_JPY_2*x_CAD_3
      + 2.378980627399 x_JPY_2*x_CNY_1 - 2.379131688155 x_JPY_2*x_CNY_3
      + 4.118748118132 x_JPY_2*x_EUR_1 - 4.118967030135 x_JPY_2*x_EUR_3
      - 4.034553224663 x_JPY_3*x_USD_1 + 4.034484169095 x_JPY_3*x_USD_2
      - 3.795132588637 x_JPY_3*x_CAD_1 + 3.794594244119 x_JPY_3*x_CAD_2
      - 2.379131688155 x_JPY_3*x_CNY_1 + 2.378980627399 x_JPY_3*x_CNY_2
      - 4.118967030135 x_JPY_3*x_EUR_1 + 4.118748118132 x_JPY_3*x_EUR_2
      + 0.240515931526 x_USD_1*x_CAD_2 -

As you can see, the problem has 5 x 3 = 15 variables. Now let’s solve this problem with all three solvers.

#### CPLEX solver

In [12]:
# Calling the CplexOptimizer bundled with qiskit
cplex_result = CplexOptimizer().solve(mdl)

# Calling the helper function on results
raw_result = get_result_arbitrage(cplex_result, c_data, cur, mdl)
cycle = raw_result['cycle']
profit = raw_result['profit']

print(
    '-'*80,
    '[CPLEX Result]',
    f'   optimal value: {cplex_result.fval:.5e}',
    f'   optimal solution: {cplex_result.x.astype(int)}',
    f'   status: {cplex_result.status.name}',
    f'   best arbitrage: {cycle}',
    f'   best gain: {100*(profit-1):.5f}%',
    '-'*80,
    sep='\n'
)

--------------------------------------------------------------------------------
[CPLEX Result]
   optimal value: 3.03309e-04
   optimal solution: [0 0 0 0 0 1 1 0 0 0 0 0 0 1 0]
   status: SUCCESS
   best arbitrage: CAD => EUR => USD => CAD
   best gain: 0.06986%
--------------------------------------------------------------------------------


In [13]:
qubo = QuadraticProgramToQubo().convert(mdl)

hamiltonian_info = qubo.to_ising()

#### QAOA solver

In [14]:
algorithm_globals.random_seed = 1234

# Creating the quantum instance
quantum_instance = QuantumInstance(Aer.get_backend('aer_simulator'),
                                   seed_simulator=algorithm_globals.random_seed,
                                   seed_transpiler=algorithm_globals.random_seed)

# Calling the qiskit's QAOA solver, we keep the number of repetitions to P=2
qaoa_mes = QAOA(quantum_instance=quantum_instance, reps=2)
qaoa = MinimumEigenOptimizer(qaoa_mes)
qaoa_result = qaoa.solve(qubo)

# Calling the helper function on results
raw_result = get_result_arbitrage(qaoa_result, c_data, cur, mdl)
cycle = raw_result['cycle']
profit = raw_result['profit']

print(
    '-'*80,
    '[QAOA Result]',
    f'   optimal value: {qaoa_result.fval:.5e}',
    f'   optimal solution: {qaoa_result.x.astype(int)}',
    f'   status: {qaoa_result.status.name}',
    f'   best arbitrage: {cycle}',
    f'   best gain: {100*(profit-1):.5f}%',
    '-'*80,
    sep='\n'
)

--------------------------------------------------------------------------------
[QAOA Result]
   optimal value: -3.03309e-04
   optimal solution: [0 0 0 0 1 0 0 0 1 0 0 0 1 0 0]
   status: SUCCESS
   best arbitrage: EUR => USD => CAD => EUR
   best gain: 0.06986%
--------------------------------------------------------------------------------


#### GNQA solver

In [15]:
# Definition of filter function
p = 8
filter_function = lambda x: np.power(x, -p)

# Execution of GNQA
GNQA = GNQA_optimizer(qubo)
GNQA.set_filtered_hamiltonian(filter_function, Omega=[0,1], epsiron=1e-8)
GNQA_result = GNQA.solve(step_max=20)

raw_result = get_result_arbitrage(GNQA_result, c_data, cur, mdl)
cycle = raw_result['cycle']
profit = raw_result['profit']


print(
    '-'*80,
    '[GNQA Result]',
    f'   optimal value: {GNQA_result.fval:.5e}',
    f'   optimal solution: {GNQA_result.x}',
    f'   status: {GNQA_result.status.name}',
    f'   best arbitrage: {cycle}',
    f'   best gain: {100*(profit-1):.5f}%',
    '-'*80,
    sep='\n'
)

--------------------------------------------------------------------------------
[GNQA Result]
   optimal value: -3.03309e-04
   optimal solution: [0 0 0 1 0 0 0 1 0 0 0 0 0 0 1]
   status: SUCCESS
   best arbitrage: USD => CAD => EUR => USD
   best gain: 0.06986%
--------------------------------------------------------------------------------


#### Interpret the solutions

As seen, by increasing the problem size from 12 ($n=4$ and $K=3$) to 15 ($n=5$, $K=3$), the solution of GNQA still coincides with the CPLEX solver, but the QAOA solver fails to solve the problem with the previous parameters ($P=2$ iterations). We refer the reader to Takeori et al. 2022<sup>3</sup> for the justification of outperforming GNQA versus QAOA on some classes of problems. 

### Summary

Potential next steps include the following:

 - Using the edge-based Hamiltonian cycles formulation and comparing it with the node-based model
 - Adding the option of starting from a specific currency
 - Modeling the transaction cost separately

### References

1. <a id='CPLEX'></a>IBM® ILOG® CPLEX® Optimization Studio. https://www.ibm.com/products/ilog-cplex-optimization-studio.

2. <a name="QAOA"></a>“Quantum Approximate Optimization Algorithm.” *Qiskit*. https://qiskit.org/documentation/tutorials/algorithms/05_qaoa.html.

3. <a name="3"></a>Mitsuharu Takeori et al. “A Gauss-Newton Based Quantum Algorithm for Combinatorial Optimization.” arXiv:2203.13939v4, June 17, 2022. https://doi.org/10.48550/arXiv.2203.

4. <a name="Currency"></a>Wanmei Soon and Heng-Qing Ye. “Currency Arbitrage Detection Using a Binary Integer Programming Model.” *International Journal of Mathematical Education in Science and Technology*, vol. 42, no. 3 (2011): 369–376. https://doi.org/10.1080/0020739X.2010.526248.

5. <a name="1qbit"></a>Gili Rosenberg. “Finding Optimal Arbitrage Opportunities Using a Quantum Annealer.” *1QBit*, 2016. https://1qbit.com/whitepaper/arbitrage/.

6. <a name="QiskitOptimization"></a>“Quadratic Programs.” *Qiskit*. https://qiskit.org/documentation/optimization/tutorials/01_quadratic_program.html.


<span style="font-size:10pt; font-weight:bold;" class = "copyright-content"> &copy; Copyright IBM Corp. 2023 <br/>This content is categorized as an “existing” IBM asset as part of the IBM Quantum Accelerator contract. </span>